<h1 style="color:#2E86C1; font-family: 'Helvetica';">robust.py</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
Implementation of the <b>Betsimas Sim</b> robust Min-Max Flow problem under budget uncertainty.
</p>

In [263]:
import os
import json
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(1) Data Preprocessing</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part:</p>

<p>- Validate files </p>
<p>- Load graph data </p>
<p>- Build graph structure </p>

In [264]:
def validate_file(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {os.path.abspath(path)}!")

In [265]:
def load_graph_data(nodes_csv: str, edges_csv: str):
    validate_file(nodes_csv)
    validate_file(edges_csv)

    Nd = pd.read_csv(nodes_csv).copy()
    Nd["node_id"] = Nd["node_id"].astype(int)

    Ed = pd.read_csv(edges_csv).copy()
    for col in ["edge_id", "u", "v"]:
        Ed[col] = Ed[col].astype(int)
    for col in ["nominal", "deviation"]:
        Ed[col] = Ed[col].astype(float)

    V = set(Nd["node_id"].tolist())
    edges = list(Ed["edge_id"])
    return Nd, Ed, V, edges

In [266]:
def build_graph_structure(Ed, V):
    edges = list(Ed["edge_id"])
    idx_by_eid = {eid: i for i, eid in enumerate(edges)}
    out_by_node = {i: [] for i in V}
    in_by_node = {i: [] for i in V}

    for _, r in Ed.iterrows():
        out_by_node[r.u].append(int(r.edge_id))
        in_by_node[r.v].append(int(r.edge_id))

    return idx_by_eid, out_by_node, in_by_node

In [267]:
def load_parameters(param_path: str) -> dict:
    if not os.path.exists(param_path):
        raise FileNotFoundError(f"Parameters not found: {param_path}")
    with open(param_path, "r") as f:
        params = json.load(f)
    return params

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(2) Optimization Model</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
Implementation of the Bertsimas-Sim budget under uncertainty model.</p>

<p>- Create model </p>
<p>- Decision variables </p>
<p>- Objective function </p>
<p>- Robust constraints </p>
<p>- Flow balance constraints </p>


In [268]:
def parse_gamma(robust_params: dict, num_edges: int) -> list[float]:
    gamma_values    = []
    gamma_raw       = robust_params.get("Gamma_values", [0])    # default is nominal case

    for g in gamma_raw:
        if isinstance(g, str) and g.lower() == "full":
            gamma_values.append(float(num_edges))       # full protection
        else:
            gamma_values.append(float(g))
    
    return gamma_values

In [269]:
def create_model():
    """Initialize a Gurobi model."""
    m = gp.Model("RobustMCF_BertsimasSim")
    m.Params.OutputFlag = 1
    return m

In [270]:
def add_decision_variables(m, edges):
    """Add binary x, global pi, and continuous rho variables."""
    x = m.addVars(edges, vtype=GRB.BINARY, name="x")
    pi = m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name="pi")
    rho = m.addVars(edges, lb=0.0, vtype=GRB.CONTINUOUS, name="rho")
    return x, pi, rho

In [271]:
def set_objective(m, Ed, idx_by_eid, edges, x, rho, pi, Gamma):
    obj = (
        gp.quicksum(Ed.loc[idx_by_eid[eid], "nominal"] * x[eid] for eid in edges)
        + Gamma * pi
        + gp.quicksum(rho[eid] for eid in edges)
    )
    m.setObjective(obj, GRB.MINIMIZE)

In [272]:
def add_robust_constraints(m, Ed, idx_by_eid, edges, x, rho, pi):
    for eid in edges:
        d_e = Ed.loc[idx_by_eid[eid], "deviation"]
        m.addConstr(pi + rho[eid] >= d_e * x[eid], name=f"robust_{eid}")

In [273]:
def add_flow_constraints(m, x, out_by_node, in_by_node, start, goal):
    """Add standard flow-balance constraints."""
    for i in out_by_node.keys():
        b_i = 1 if i == start else (-1 if i == goal else 0)
        m.addConstr(
            gp.quicksum(x[eid] for eid in out_by_node[i]) -
            gp.quicksum(x[eid] for eid in in_by_node[i]) == b_i,
            name=f"flow_{i}"
        )

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(3) Data Posprocessing</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data postprocessing part:</p>

<p>- Extract decision variable values </p>
<p>- Build successor map </p>
<p>- Reconstruct path </p>
<p>- Compute costs </p>
<p>- Extract solution </p>
<p>- Map path edges </p>
<p>- Build breakdown table </p>
<p>- Add summary rows </p>
<p>- Save results to csv </p>
<p>- Export results to directory </p>


In [274]:
def extract_decision_variables(m, x, rho, pi, Ed):
    edges = list(Ed["edge_id"])
    x_sol = {eid: int(round(x[eid].X)) for eid in edges if x[eid].X > 0.5}
    rho_vals = {eid: float(rho[eid].X) for eid in edges}
    pi_val = float(pi.X)
    obj_val = float(m.ObjVal)
    return x_sol, rho_vals, pi_val, obj_val

In [275]:
def build_successor_map(Ed, x_sol):
    succ = {}
    for _, r in Ed.iterrows():
        if int(r.edge_id) in x_sol:
            succ[int(r.u)] = int(r.v)
    return succ

In [276]:
def reconstruct_path(succ, start, goal):
    path_nodes = [start]
    cur = start
    visited = {start}

    while cur in succ:
        nxt = succ[cur]
        path_nodes.append(nxt)
        if nxt in visited:
            print("[Warning] Cycle detected in path.")
            break
        visited.add(nxt)
        cur = nxt
        if cur == goal:
            break
    return path_nodes

In [277]:
def compute_costs(Ed, idx_by_eid, chosen_eids, rho_vals, pi_val, Gamma):
    nominal_cost    = float(sum(Ed.loc[idx_by_eid[eid], "nominal"] for eid in chosen_eids))
    sum_rho         = float(sum(rho_vals[eid] for eid in chosen_eids))
    robust_obj      = nominal_cost + Gamma * pi_val + sum_rho
    return nominal_cost, sum_rho, robust_obj

In [278]:
def extract_solution(m, x, rho, pi, Ed, idx_by_eid, start, goal, Gamma):
    x_sol, rho_vals, pi_val, obj_val    = extract_decision_variables(m, x, rho, pi, Ed)
    chosen_eids                         = list(x_sol.keys())
    succ                                = build_successor_map(Ed, x_sol)
    path_nodes                          = reconstruct_path(succ, start, goal)
    nominal_cost, sum_rho, robust_obj   = compute_costs(Ed, idx_by_eid, chosen_eids, rho_vals, pi_val, Gamma)

    return {
        "x_sol"         : x_sol,
        "rho_vals"      : rho_vals,
        "pi_val"        : pi_val,
        "obj_val"       : obj_val,
        "chosen_eids"   : chosen_eids,
        "path_nodes"    : path_nodes,
        "nominal_cost"  : nominal_cost,
        "sum_rho"       : sum_rho,
        "robust_obj"    : robust_obj,
    }

In [279]:
def map_path_edges(Ed, path_nodes, chosen_eids):
    path_edge_ids = []
    for i in range(len(path_nodes) - 1):
        from_node, to_node = path_nodes[i], path_nodes[i + 1]
        row = Ed[
            (Ed["u"] == from_node)
            & (Ed["v"] == to_node)
            & (Ed["edge_id"].isin(chosen_eids))
        ]
        eid = int(row.iloc[0]["edge_id"]) if not row.empty else None
        path_edge_ids.append(eid)
    return [eid for eid in path_edge_ids if eid is not None]

In [280]:
def build_breakdown_table(Ed, idx_by_eid, path_edge_ids, pi_val):
    breakdown_rows = []
    for idx, eid in enumerate(path_edge_ids):
        edge_row = Ed.loc[idx_by_eid[eid]]
        deviation = edge_row["deviation"]
        covered_by_pi = deviation <= pi_val + 1e-9
        rho_implied = max(deviation - pi_val, 0.0)

        breakdown_rows.append({
            "order": idx + 1,
            "edge_id": eid,
            "u": int(edge_row["u"]),
            "v": int(edge_row["v"]),
            "nominal": edge_row["nominal"],
            "deviation": deviation,
            "covered_by_pi": covered_by_pi,
            "rho_implied": rho_implied,
            "robust_cost_component": rho_implied,
        })
    return pd.DataFrame(breakdown_rows)

In [281]:
def add_summary_rows(df_breakdown, nominal_cost, robust_obj):
    extra_rows = pd.DataFrame([
        {"order": None, "edge_id": None, "u": None, "v": None,
         "nominal": None, "deviation": None,
         "covered_by_pi": None, "rho_implied": None, "robust_cost_component": None},
        {"order": "TOTAL_NOMINAL_COST", "nominal": nominal_cost},
        {"order": "TOTAL_ROBUST_OBJECTIVE", "nominal": robust_obj}
    ])
    return pd.concat([df_breakdown, extra_rows], ignore_index=True)

In [282]:
def save_results_to_csv(out_dir, df_breakdown, path_nodes):
    breakdown_csv           = os.path.join(out_dir, "robust_solution_breakdown.csv")
    robust_path_nodes_csv   = os.path.join(out_dir, "robust_path_nodes.csv")

    df_breakdown.to_csv(breakdown_csv, index=False)
    pd.DataFrame({"path_node_id": path_nodes}).to_csv(robust_path_nodes_csv, index=False)

    print(f"\nBreakdown saved to: {breakdown_csv}")
    print(f"Path saved to: {robust_path_nodes_csv}")
    return breakdown_csv, robust_path_nodes_csv

In [283]:
def export_results(Ed, idx_by_eid, path_nodes, chosen_eids, pi_val,
                   nominal_cost, robust_obj, Gamma, edges_csv):
    out_dir                 = os.path.dirname(edges_csv)
    path_edge_ids           = map_path_edges(Ed, path_nodes, chosen_eids)
    df_breakdown            = build_breakdown_table(Ed, idx_by_eid, path_edge_ids, pi_val)
    df_breakdown            = add_summary_rows(df_breakdown, nominal_cost, robust_obj)
    breakdown_csv, path_csv = save_results_to_csv(out_dir, df_breakdown, path_nodes)
    return breakdown_csv, path_csv

In [ ]:
def plot_robust_path_overlay(forecast_npy, nodes_csv, path_nodes, nominal_cost, robust_obj, output_dir):
    """Plot the robust path overlay directly after optimization."""
    # Load cost field and node coordinates
    cost_field = np.load(forecast_npy)
    nodes = pd.read_csv(nodes_csv)
    coords = nodes[["x", "y"]].to_numpy()

    # Convert path node IDs to coordinates
    id_to_idx = {int(row.node_id): i for i, row in nodes.iterrows()}
    path_xy = np.array([coords[id_to_idx[nid]] for nid in path_nodes])


    # Plot
    plt.figure(figsize=(8, 8))
    cost_field = cost_field.T
    im = plt.imshow(cost_field, origin="lower", cmap="viridis")
    plt.colorbar(im, label="Forecast Cost")

    plt.plot(path_xy[:, 0], path_xy[:, 1], color="blue", linestyle="--", linewidth=2, label="Robust Path")
    plt.scatter(path_xy[0, 0], path_xy[0, 1], color="green", s=200, label="Start")
    plt.scatter(path_xy[-1, 0], path_xy[-1, 1], color="red", s=200, label="Goal")

    #plt.grid(True, linestyle='--', alpha=0.5)
    plt.gca().set_aspect('equal', adjustable='box')

    plt.title(f"Robust Path Overlay\nNominal: {nominal_cost:.2f} | Robust: {robust_obj:.2f}")
    plt.legend(loc="upper left")

    os.makedirs(output_dir, exist_ok=True)
    out_png = os.path.join(output_dir, "robust_path_overlay.png")
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()
    print(f"Saved inline robust path visualization: {out_png}")

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(4) Wrapper & Main</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
The data preprocessing part consists of two parts:</p>

In [285]:
def robust_shortest_path(nodes_csv, edges_csv, start, goal, Gamma=0.0, output_dir=None, parameters=None):
    """Main orchestration of the robust shortest-path pipeline."""
    # --- 1. Load and prepare data ---
    Nd, Ed, V, edges = load_graph_data(nodes_csv, edges_csv)
    idx_by_eid, out_by_node, in_by_node = build_graph_structure(Ed, V)

    # --- 2. Build model ---
    m = create_model()
    x, pi, rho = add_decision_variables(m, edges)
    set_objective(m, Ed, idx_by_eid, edges, x, rho, pi, Gamma)
    add_robust_constraints(m, Ed, idx_by_eid, edges, x, rho, pi)
    add_flow_constraints(m, x, out_by_node, in_by_node, start, goal)

    # --- 3. Solve model ---
    m.optimize()

    # --- 4. Extract results ---
    result = extract_solution(m, x, rho, pi, Ed, idx_by_eid, start, goal, Gamma)

    # --- 5. Export results to CSV ---
    if output_dir is not None:
        os.makedirs(output_dir, exist_ok=True)
        edges_out_path = os.path.join(output_dir, "edges.csv")
    else:
        edges_out_path = edges_csv

    breakdown_csv, path_csv = export_results(
        Ed, idx_by_eid, result["path_nodes"], result["chosen_eids"],
        result["pi_val"], Gamma, result["nominal_cost"], result["robust_obj"],
        edges_out_path
    )

    result["breakdown_csv"] = breakdown_csv
    result["path_csv"] = path_csv

    # --- 6. Inline Visualization (wired to existing function) ---
    try:
        forecast_npy = edges_csv.replace("edges.csv", "forecast.npy")

        # Allow visualization control from JSON
        visualize = parameters.get("general", {}).get("visualize", True) if parameters else True

        if visualize and os.path.exists(forecast_npy):
            plot_robust_path_overlay(
                forecast_npy=forecast_npy,
                nodes_csv=nodes_csv,
                path_nodes=result["path_nodes"],
                nominal_cost=result["nominal_cost"],
                robust_obj=result["robust_obj"],
                output_dir=os.path.dirname(edges_csv)
            )
        elif visualize:
            print("forecast.npy not found — skipping inline visualization.")
        else:
            print("Visualization disabled in parameters.json.")
    except Exception as e:
        print(f"Inline visualization failed: {e}")

    # --- 7. Return results dictionary ---
    return result

In [286]:
if __name__ == "__main__":
        
        dir = "/Users/philippstockerl/BachelorThesis/Data/master78/seed_39622_grid100_Stable_ls30.0_var1.0_nug0.05/"
        N = 100
        offset = 10
        start = offset * N + offset
        goal  = (N - offset - 1) * N + (N - offset - 1)

        params = load_parameters("parameters.json")
        robust_params = params.get("robust", {})

        edges_df = pd.read_csv(f"{dir}/edges.csv")
        num_edges = len(edges_df)

        gamma_values = parse_gamma(robust_params, num_edges)

        output_dir = os.path.dirname(dir)
        for Gamma in gamma_values:
            subfolder = os.path.join(output_dir, f"robust_Gamma_{Gamma}")
            os.makedirs(subfolder, exist_ok=True)

            result = robust_shortest_path(
                nodes_csv=f"{dir}/nodes.csv",
                edges_csv=f"{dir}/edges.csv",
                start=0,
                goal=9999,
                Gamma=Gamma,
                output_dir=subfolder,
                parameters=params,
            )


        print("\n=== Robust Result ===")
        print(f"Robust objective: {result['robust_obj']:.3f}")
        print(f"Nominal cost on path: {result['nominal_cost']:.3f}")
        print(f"Gamma * pi: {result['pi_val']:.3f}")
        print(f"Sum rho: {result['sum_rho']:.3f}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G222)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49600 rows, 79201 columns and 198000 nonzeros
Model fingerprint: 0xdef64b62
Variable types: 39601 continuous, 39600 integer (39600 binary)
Coefficient statistics:
  Matrix range     [7e-07, 1e+00]
  Objective range  [4e-03, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 39602 rows and 39607 columns
Presolve time: 0.09s
Presolved: 9998 rows, 39594 columns, 79188 nonzeros
Variable types: 0 continuous, 39594 integer (39594 binary)
Found heuristic solution: objective 113.9678276
Found heuristic solution: objective 113.6636711

Root relaxation: objective 9.308874e+01, 8761 iterations, 0.71 seconds (2.86 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   